In [1]:
import sys

path_src = '../src'
sys.path.append(path_src)
import importlib
import SpatialAnalysis as SpAn
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

path_src = '../../tysserand2/'


Script loaded!


In [8]:
importlib.reload(SpAn)
coords = pd.read_csv('../data/simulation_coordiantes/example_01.csv',sep=',')
coords.head()

Script loaded!


,0,1,Type
0,3699.797871,1449.850342,A
1,4211.098154,3074.756329,B
2,1745.432210,4488.494023,A
3,1359.813983,3621.950668,A
4,3434.859928,1133.231468,A


# Geometric Features

In [142]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

# Define all the individual metric functions
def compute_abundance(df, phenotype_col='celltypes'):
    abnd = df[phenotype_col].value_counts()
    abnd.index = 'abundance_' + abnd.index
    return abnd.to_dict()

def compute_proportions(df, phenotype_col='celltypes'):
    prop = df[phenotype_col].value_counts(normalize=True)
    prop.index = 'prop_' + prop.index
    return prop.to_dict()

def compute_area(df):
    return {'area':df.iloc[:,0].max() - df.iloc[:,0].min() * df.iloc[:,1].max() - df.iloc[:,1].min()}

def compute_density(df, area, phenotype_col='celltypes'):
    count = df[phenotype_col].value_counts()
    count = count / area
    count.index = f'density_in_{area}_' + count.reset_index()[phenotype_col]
    return count.to_dict()

def compute_distribution_uniformity(df, grid_size, phenotype_col='celltypes'):
    area_value = next(iter(compute_area(df).values()))
    if area_value > grid_size:
        df['grid_x'] = pd.cut(df['x'], bins=int(df['x'].max() / grid_size))
        df['grid_y'] = pd.cut(df['y'], bins=int(df['y'].max() / grid_size))
        grid_counts = df.groupby(['grid_x', 'grid_y', phenotype_col]).size().unstack(fill_value=0).std()
        grid_counts = pd.Series(grid_counts)
        grid_counts.index = f'uniformity_{grid_size}_' + grid_counts.index
        distrib = grid_counts.to_dict()
        distrib[f'uniformity_{grid_size}_mean'] = grid_counts.std().mean()
        return distrib
        return 
    else:
        return {}

def compute_spatial_entropy(df, phenotype_col='celltypes'):
    counts = df[phenotype_col].value_counts()
    probabilities = counts / counts.sum()
    return {'spatial_entropy': entropy(probabilities)}

def compute_area_fraction(df, phenotype_col='celltypes'):
    total_area = next(iter(compute_area(df).values()))
    count = df[phenotype_col].value_counts()
    count = count * (1 / total_area)
    count.index = f'area_fraction_' + count.index
    return count.to_dict()  # Assuming each cell type occupies an equal portion of the total area


# Aggregating features into a single row DataFrame
def compute_geometric_features(df, Density_area, grid_size, phenotype_col='celltypes'):
    features = pd.DataFrame(index=[0])  # Create a DataFrame with a single row
    
    abundance = compute_abundance(df, phenotype_col)
    proportions = compute_proportions(df, phenotype_col)
    area = compute_area(df)
    density = compute_density(df, Density_area, phenotype_col)
    distribution_uniformity = compute_distribution_uniformity(df, grid_size, phenotype_col)
    spatial_entropy = compute_spatial_entropy(df, phenotype_col)
    area_fraction = compute_area_fraction(df, phenotype_col)


    # Combine dictionaries
    combined_dict = {**abundance, **proportions, **area, **density, **distribution_uniformity, **spatial_entropy, **area_fraction}

    # Create DataFrame
    features = pd.DataFrame([combined_dict])

    return features


In [132]:




# Example usage with the same DataFrame:
df = pd.DataFrame({
    'x': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'y': [1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
    'celltypes': ['A', 'B', 'A', 'B', 'C', 'C', 'A', 'B', 'C', 'A'],
    'sample': ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C']
})


In [153]:
sample_col = 'sample'
Density_area = 2
grid_size = 2
phenotype_col='celltypes'

def compute_geometric_features_samples(df, Density_area, grid_size, phenotype_col=phenotype_col, sample_col=sample_col):
    AllFeatures = {}
    for sample in df[sample_col].unique():
        df_sample = df.loc[df[sample_col] == sample ,:]
        x = compute_geometric_features(df_sample, Density_area = Density_area, grid_size = grid_size, phenotype_col= phenotype_col)
        AllFeatures[sample] = x
    return pd.concat(AllFeatures,axis=0)

In [154]:
compute_geometric_features_samples(df, Density_area, grid_size, phenotype_col=phenotype_col, sample_col=sample_col)

,,abundance_A,abundance_B,prop_A,prop_B,area,density_in_2_A,density_in_2_B,spatial_entropy,area_fraction_A,area_fraction_B,abundance_C,prop_C,density_in_2_C,area_fraction_C
A,0,2,2.0,0.50,0.50,1,1.0,1.0,0.693147,2.000000,2.000000,NaN,NaN,NaN,NaN
B,0,1,1.0,0.25,0.25,-15,0.5,0.5,1.039721,-0.066667,-0.066667,2.0,0.5,1.0,-0.133333
C,0,1,NaN,0.50,NaN,-40,0.5,NaN,0.693147,-0.025000,NaN,1.0,0.5,0.5,-0.025000
